In [ ]:
#| default_exp inspecttools

# inspecttools
> LLM tools for inspecting the symbol table and importing modules
- skip_exec: true

This module provides **LLM tools** to dynamically inspect source code, types, and module capabilities. Functions take *string* arguments (dotted symbol paths) rather than Python objects because LLM tool interfaces can only pass serializable values—not live Python references.

In [ ]:
#| export
from fastcore.utils import *
from fastcore.meta import delegates
import inspect, re, sys, ast, builtins, os, linecache
from importlib import import_module

from toolslm.xml import *

In [ ]:
from IPython.display import display,Markdown
import textwrap

## Helpers

In [ ]:
#| export
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = inspect.currentframe().f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
#| export
@llmtool
def importmodule(
    mod: str, # The module to import (e.g. 'torch.nn.functional')
    caller_symbol:str = '__msg_id'  # The name of the special variable to find the correct caller namespace
):
    """Import a module into the caller's global namespace so it's available for `symsrc`, `symval`, `symdir`, etc.
    Use this before inspecting or using symbols from modules not yet imported."""
    g = _find_frame_dict(caller_symbol)
    import_module(mod)
    g[mod.split('.')[0]] = import_module(mod.split('.')[0])

`importmodule` lets the LLM dynamically import modules by name. Here we import `fastcore.utils` and verify it's available.

In [ ]:
importmodule('fastcore.utils')
fastcore.__version__

'1.12.5'

In [ ]:
#| export
class SymbolNotFound(Exception):
    def __repr__(self): return f"SymbolNotFound({self.args[0]})"
    __str__ = __repr__

_last = None

def resolve(
    sym: str  # Dotted symbol path, with optional [n] indexing, e.g. "module.attr.subattr[1]" or "_last" for previous result
):
    """Resolve a dotted symbol string to its Python object, with optional [n] indexing.
    Sets global `_last` to the resolved object for chaining.
    Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist"""
    global _last
    if (sym := sym.strip()) == '_last': return _last
    g = _find_frame_dict('__msg_id')
    if match := re.match(r'^(\w+)\[(\d+)\]$', sym):
        attr, idx = match.groups()
        parts, _last = ['_last'], _last[int(idx)] if attr == '_last' else g[attr][int(idx)]
    else: parts = re.split(r'\.(?![^\[]*\])', sym)
    try: obj = _last if parts[0] == '_last' else g[parts[0]]
    except KeyError: raise SymbolNotFound(f"Symbol '{parts[0]}' not found. Consider using `importmodule` first.")
    for part in parts[1:]:
        match = re.match(r'(\w+)\[(\d+)\]$', part)
        if match:
            attr, idx = match.groups()
            obj = getattr(obj, attr)[int(idx)]
        else: obj = getattr(obj, part)
    _last = obj
    return obj

`resolve` navigates dotted paths like `"a.argfirst"` to reach the actual Python object.

In [ ]:
a = fastcore.utils.L(1)
resolve('a.argfirst')

<bound method L.argfirst of [1]>

It also sets `_last` for chaining. Since it's used internally by all `get*` tools in this module, the tools all set `_last` too.

In [ ]:
_last

<bound method L.argfirst of [1]>

It works on both objects and classes:

In [ ]:
resolve('fastcore.utils.L.argfirst')

<function fastcore.foundation.L.argfirst(self: fastcore.foundation.L, f, negate=False)>

## Symbol info

In [ ]:
#| export
def _src_from_lines(lines, start):
    "Extract a single definition from lines starting at start (0-indexed)"
    src = ''.join(lines[start:])
    try: tree = ast.parse(src)
    except SyntaxError: return None
    if not tree.body: return None
    return ''.join(lines[start:start + tree.body[0].end_lineno])

In [ ]:
#| export
@llmtool
def symsrc(
    sym: str  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
):
    """Get the source code for a symbol.

    Examples:

    - `symsrc("Interval")` -> source code of Interval class if it's already imported
    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `symnth("module.dispatcher.funcs", n) then symsrc("_last")`"""
    try: obj = resolve(sym)
    except SymbolNotFound as e: return str(e)
    if isinstance(obj, type) or callable(obj): pass
    elif hasattr(obj, '__module__') and not inspect.ismodule(obj): obj = obj.__class__
    try: fname = inspect.getfile(obj)
    except (OSError, TypeError): fname = "<session>"
    try: return f"File: {fname}\n\n{inspect.getsource(obj)}"
    except (OSError, TypeError): pass
    name = getattr(obj, '__name__', None)
    if not name: raise OSError(f"Cannot get source for {sym}")
    pat = rf'^(class|def)\s+{name}\b'
    for fname, (_, _, lines, _) in linecache.cache.items():
        src = ''.join(lines)
        if match := re.search(pat, src, re.MULTILINE):
            start = src[:match.start()].count('\n')
            if extracted := _src_from_lines(lines, start): return f"File: {fname}\n\n{extracted}"
    raise OSError(f"Source for {name} not found")

`symsrc` retrieves the source code of any symbol by path—essential for letting the LLM understand how functions work.

In [ ]:
print(symsrc('a.argfirst'))

File: /Users/jhoward/aai-ws/fastcore/fastcore/foundation.py

@patch
@curryable
def argfirst(self:L, f, negate=False):
    "Return index of first matching item"
    if negate: f = not_(f)
    return first(i for i,o in self.enumerate() if f(o))



In [ ]:
class B:
    def a(): ...
b = B()
print(symsrc('b'))

File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_33003/2711726673.py

class B:
    def a(): ...



In [ ]:
def f():
    "testing"
    return 1
print(symsrc('f'))

File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_33003/242320393.py

def f():
    "testing"
    return 1



In [ ]:
from toolslm import xml

In [ ]:
print(symsrc('xml')[:200])

File: /Users/jhoward/aai-ws/toolslm/toolslm/xml.py

# AUTOGENERATED! DO NOT EDIT! File to edit: ../00_xml.ipynb.

# %% auto #0
__all__ = ['doctype', 'json_to_xml', 'get_mime_text', 'cell2out', 'cell2x


In [ ]:
#| export
@llmtool
def symtype(
    syms: str  # Comma separated str list of dotted symbol paths (e.g `'Interval,a'` or `'sympy.sets.sets.Interval'`); "_last" for prev result
):
    """Get the type of a symbol and set `_last`.

    Examples:

    - `symtype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `symtype("doesnotexist")` -> `'SymbolNotFound`
    - `symtype("_last")` -> type of previous result"""
    def f(o):
        try: return type(resolve(o))
        except SymbolNotFound as e: return str(e)
    return [f(o) for o in re.split(r'\,\s*', syms)]

`symtype` returns the type of a symbol—useful for the LLM to understand what kind of object it's dealing with.

In [ ]:
symtype('a.argfirst')

[method]

In [ ]:
symtype('fffaa,b')

["SymbolNotFound(Symbol 'fffaa' not found. Consider using `importmodule` first.)",
 __main__.B]

In [ ]:
#| export
@llmtool
def symval(
    syms: str  # Comma separated str list of dotted symbol paths (e.g `Interval` or `sympy.sets.sets.Interval`); "_last" for prev result
):
    """List of repr of symbols' values.

    Examples:
    
    - `symval("sympy.sets.sets.Interval")` -> `[<class 'sympy.sets.sets.Interval'>]`
    - `symval("some_dict.keys")` -> `[dict_keys([...])]`
    - `symval("a,notexist")` -> `['foo','SymbolNotFound']`"""
    def f(o):
        try: return repr(resolve(o))
        except SymbolNotFound as e: return str(e)
    return [f(o) for o in re.split(r'\,\s*', syms)]

`symval` returns the `repr()` of a symbol's value—handy for inspecting data without needing to execute arbitrary code.

In [ ]:
a

[1]

In [ ]:
symval('a,foofoo')

['[1]',
 "SymbolNotFound(Symbol 'foofoo' not found. Consider using `importmodule` first.)"]

In [ ]:
#| export
def symtype_val(
    syms: str  # Comma separated str list of dotted symbol paths (e.g `Interval` or `sympy.sets.sets.Interval`); "_last" for prev result
):
    """List of 2-ple of (type,repr) of symbols' values.

    Examples:
    
    - `symtype_val("a,c,notexist")` -> `[(<class 'str'>,'foo'),(<class 'int'>,1), 'SymbolNotFound']`"""
    def f(o):
        try: r = resolve(o)
        except SymbolNotFound as e: return 'SymbolNotFound'
        return (type(r), repr(r))
    return [f(o) for o in re.split(r'\,\s*', syms)]

In [ ]:
symtype_val('a,b,foofoo')

[(fastcore.foundation.L, '[1]'),
 (__main__.B, '<__main__.B object>'),
 'SymbolNotFound']

In [ ]:
#| export
@llmtool
def symdir(
    sym: str,  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
    exclude_private: bool=False # Filter out attrs starting with "_"
):
    """Get dir() listing of a symbol's attributes and set `_last`. E.g: `symdir("sympy.Interval")` -> `['__add__', '__and__', ...]`"""
    res = dir(resolve(sym))
    if not exclude_private: return res
    return [o for o in res if o[0]!='_']

`symdir` lists all attributes of an object (i.e it calls `dir()`. Here we filter out private names to see the public API of an `L` list.

In [ ]:
' '.join(symdir('a', exclude_private=True))

'accumulate append argfirst argwhere attrgot batched clear combinations compress concat copy copy count cycle dropwhile enumerate extend filter flatmap flatten groupby index insert itemgot items map map_dict map_first map_zip map_zipwith pairwise partition permutations pop product range reduce remove renumerate reverse rstarargfirst rstarargwhere rstardropwhile rstarfilter rstarmap rstarpartition rstarreduce rstarsorted rstartakewhile setattrs shuffle sort sorted split splitlines starargfirst starargwhere stardropwhile starfilter starmap starpartition starreduce starsorted startakewhile sum takewhile unique val2idx zip zipwith'

In [ ]:
#| export
@llmtool
def symnth(
    sym: str,  # Dotted symbol path to a dict or object with .values()
    n: int     # Index into the values (0-based)
):
    """Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `symnth("dispatcher.funcs", 12)` -> 13th registered function
    - `symnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler"""
    global _last
    _last = list(resolve(sym).values())[n]
    return _last

`symnth` extracts the nth value from a dict (or anything with `.values()`).

In [ ]:
handlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))
symnth('handlers', 0)

<function __main__.<lambda>(x)>

Combined with `_last`, this lets the LLM drill into registries of handlers/dispatchers and then inspect their source.

In [ ]:
symsrc('_last')

'File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_33003/340145363.py\n\nhandlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))\n'

In [ ]:
#| export
@llmtool
def symlen(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    "Returns the length of the given symbol"
    return len(resolve(sym))

In [ ]:
#| export
@llmtool
def symslice(
    sym: str,   # Dotted symbol path or "_last" for previous result
    start: int, # Starting index for slice
    end: int    # Ending index for slice
):
    "Returns the contents of the symbol from the given start to the end."
    try: return resolve(sym)[start:end]
    except Exception as e: return f'Error: {e}'

In [ ]:
a = ['a', 'b', 'c', 'd']
symslice('a', 1, 3)

['b', 'c']

On failure we get a str error:

In [ ]:
symslice('resolve', 0, 1)

"Error: 'function' object is not subscriptable"

In [ ]:
#| export
@llmtool
def symsearch(
    sym:str,      # Dotted symbol path or "_last" for previous result
    term:str,     # Search term (exact string or regex pattern)
    regex:bool=True,  # If True, regex search; if False, exact match
    flags:int=0   # Regex flags (e.g., re.IGNORECASE)
):
    """Search contents of symbol, which is assumed to be str for regex, or iterable for non-regex.
    Regex mode returns (match, start, end) tuples; otherwise returns (item, index) tuples"""
    if regex: return str([(m.group(), m.start(), m.end()) for m in re.finditer(term, resolve(sym), flags)])
    else: return str([(x, i) for i, x in enumerate(resolve(sym)) if x == term])

In [ ]:
symsearch('a', 'c', regex=False), symsearch('a', 'z', regex=False)

("[('c', 2)]", '[]')

In [ ]:
text = "The quick brown fox jumps over 3 lazy dogs and 12 cats"

In [ ]:
symsearch('text', r'\d+', regex=True)

"[('3', 31, 32), ('12', 47, 49)]"

In [ ]:
symsearch('text', r'\b[aeiou]\w*', regex=True, flags=re.IGNORECASE)

"[('over', 26, 30), ('and', 43, 46)]"

In [ ]:
#| export
@llmtool
def symset(
    val: str  # Value to assign to _ai_sym
):
    "Set _ai_sym to the given value"
    _find_frame_dict('__msg_id')['_ai_sym'] = val

In [ ]:
symset('Otters are awesome!'); _ai_sym

'Otters are awesome!'

## Symbol context

In [ ]:
#| export
@llmtool
@delegates(sym2folderctx)
def symfiles_folder(
    sym:str,      # Dotted symbol path or "_last" for previous result
    **kwargs
):
    "Return XML context of files in the folder containing `sym`'s definition"
    try: s = resolve(sym)
    except SymbolNotFound as e: return str(e)
    return sym2folderctx(s, **kwargs)

In [ ]:
# print(symfiles_folder('xml'))

In [ ]:
#| export
@llmtool
@delegates(sym2pkgctx)
def symfiles_package(
    sym:str,      # Dotted symbol path or "_last" for previous result
    **kwargs
):
    "Return XML context of all files in `sym`'s top-level package"
    try: s = resolve(sym)
    except SymbolNotFound as e: return str(e)
    return sym2pkgctx(s, **kwargs)

In [ ]:
# print(symfiles_package('xml'))